### Import necessary libraries, set options

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import patsy
import seaborn as sns
import statsmodels.api as sm
import warnings

from statsmodels.formula.api import glm

pd.set_option('display.max_columns', 125)
warnings.filterwarnings("ignore")

### Read in pickled datasets

In [ ]:
path_to_diss = os.path.join("path/to/dissertation/chapter")

path_to_data = os.path.join(path_to_diss, "path/to/data/processed-data")

##### Between-individual

In [ ]:
data_final = pd.read_pickle(path_to_data + "/pkl/data_final.pkl")
print(len(data_final))
data_final.head()

In [ ]:
data_final_coop = pd.read_pickle(path_to_data + "/pkl/data_final_coop.pkl")
print(len(data_final_coop))
data_final_coop.head()

In [ ]:
data_pid_version = pd.read_pickle(path_to_data + "/pkl/data_pid_version.pkl")
print(len(data_pid_version))
data_pid_version.head()

In [ ]:
data_pid = pd.read_pickle(path_to_data + "/pkl/data_pid.pkl")
print(len(data_pid))
data_pid.head()

##### Between-session

In [ ]:
session_data = pd.read_pickle(path_to_data + "/pkl/session_data.pkl")
print(len(session_data))
session_data.head()

In [ ]:
session_data_coop = pd.read_pickle(path_to_data + "/pkl/session_data_coop.pkl")
print(len(session_data_coop))
session_data_coop.head()

In [ ]:
session_data_pid_version = pd.read_pickle(path_to_data + "/pkl/session_data_pid_version.pkl")
print(len(session_data_pid_version))
session_data_pid_version.head()

##### Within-individual

In [ ]:
data_within = pd.read_pickle(path_to_data + "/pkl/data_within.pkl")
print(len(data_within))
data_within.head()

In [ ]:
data_within_coop = pd.read_pickle(path_to_data + "/pkl/data_within_coop.pkl")
print(len(data_within_coop))
data_within_coop.head()

In [ ]:
data_within_pid = pd.read_pickle(path_to_data + "/pkl/data_within_pid.pkl")
print(len(data_within_pid))
data_within_pid.head()

##### Within-session

In [ ]:
session_data_within = pd.read_pickle(path_to_data + "/pkl/session_data_within.pkl")
print(len(session_data_within))
session_data_within.head()

In [ ]:
session_data_within_coop = pd.read_pickle(path_to_data + "/pkl/session_data_within_coop.pkl")
print(len(session_data_within_coop))
session_data_within_coop.head()

In [ ]:
session_data_within_pid = pd.read_pickle(path_to_data + "/pkl/session_data_within_pid.pkl")
print(len(session_data_within_pid))
session_data_within_pid.head()

### Generate within-subjects interaction term

In [ ]:
data_within['game1_int'] = data_within['earned_v1'] * data_within['equal_v1']
data_within['game2_int'] = data_within['earned_v2'] * data_within['equal_v2']
data_within['delta_int'] = data_within['game2_int'] - data_within['game1_int']

In [ ]:
session_data_within['game1_int'] = session_data_within['earned1'] * session_data_within['equal1']
session_data_within['game2_int'] = session_data_within['earned2'] * session_data_within['equal2']
session_data_within['delta_int'] = session_data_within['game2_int'] - session_data_within['game1_int']

### Tie formation

##### Overview

In [ ]:
print(np.mean(data_final.addChoice))
print()

print(data_final.groupby('version')['addChoice'].mean())
print()

print(data_final.groupby('round')['addChoice'].mean())
print()

In [ ]:
print(data_final.groupby('condition')['addChoice'].mean())
print()

print(data_final.groupby(['version', 'condition'])['addChoice'].mean())

##### Figures

In [ ]:
data_final.groupby(['round', 'version']).mean()['addChoice'].unstack().plot(style = ['-', '--'], color = 'black')
plt.xlabel('Round')
plt.ylabel('Average Add Tie Choice')
plt.title('Average Add Tie Choice by Round')
plt.legend(['1st  PG game', '2nd PG game'])
plt.xticks(np.arange(1, 10, step=1))

plt.savefig(os.path.join(path_to_diss, "paper/figures/appendices/figureA6.1.png"))

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (15, 5))
clrs = ['royalblue', 'orange', 'limegreen', 'tomato']

for i in range(3):
    if i == 0:
        temp = data_final[
            data_final['version'] == 1].groupby(['round',
                                                 'condition']).mean()['addChoice'].unstack().plot(color = clrs, 
                                                                                                  ax = ax[i])
        ax[i].set_title('First game only')
        ax[i].set_xticks(np.arange(1, 9, step = 1))
    elif i == 1:
        temp = data_final[
            data_final['version'] == 2].groupby(['round',
                                                 'condition']).mean()['addChoice'].unstack().plot(color = clrs,
                                                                                                  ax = ax[i])
        ax[i].set_title('Second game only')
        ax[i].set_xticks(np.arange(1, 10, step = 1))
    else:
        temp = data_final[
            data_final['round'] < 9].groupby(['round',
                                              'condition']).mean()['addChoice'].unstack().plot(color = clrs,
                                                                                               ax = ax[i])
        ax[i].set_title('Pooled')
        ax[i].set_xticks(np.arange(1, 9, step = 1))
    ax[i].set_xlabel('Round')
    ax[i].set_ylabel('Average Add Tie Choice')
    ax[i].set_yticks(np.arange(0.45, 0.85, step = 0.05))
    ax[i].legend(['EE', 'EU', 'RE', 'RU'])
    ax[i].set(ylim=(0.45, 0.80))
    
fig.savefig(os.path.join(path_to_diss, "paper/figures/appendices/figureA6.2.png"), 
            bbox_inches = 'tight', 
            pad_inches = 0.25)

In [ ]:
def get_condition_pairs(row):
    if   row.condition_v1 == 'ru' and row.condition_v2 == 'ru': return 'RU to RU'
    elif row.condition_v1 == 'ru' and row.condition_v2 == 'ee': return 'RU to EE'
    elif row.condition_v1 == 'ru' and row.condition_v2 == 're': return 'RU to RE'
    elif row.condition_v1 == 'ru' and row.condition_v2 == 'eu': return 'RU to EU'
    elif row.condition_v1 == 'ee' and row.condition_v2 == 'ru': return 'EE to RU'
    elif row.condition_v1 == 'ee' and row.condition_v2 == 'ee': return 'EE to EE'
    elif row.condition_v1 == 'ee' and row.condition_v2 == 're': return 'EE to RE'
    elif row.condition_v1 == 'ee' and row.condition_v2 == 'eu': return 'EE to EU'
    elif row.condition_v1 == 're' and row.condition_v2 == 'ru': return 'RE to RU'
    elif row.condition_v1 == 're' and row.condition_v2 == 'ee': return 'RE to EE'
    elif row.condition_v1 == 're' and row.condition_v2 == 're': return 'RE to RE'
    elif row.condition_v1 == 're' and row.condition_v2 == 'eu': return 'RE to EU'
    elif row.condition_v1 == 'eu' and row.condition_v2 == 'ru': return 'EU to RU'
    elif row.condition_v1 == 'eu' and row.condition_v2 == 'ee': return 'EU to EE'
    elif row.condition_v1 == 'eu' and row.condition_v2 == 're': return 'EU to RE'
    elif row.condition_v1 == 'eu' and row.condition_v2 == 'eu': return 'EU to EU'
    
def get_change_in_add_choice(row):
    return row.addChoice_v2 - row.addChoice_v1

In [ ]:
temp = data_final[['session_no', 'pid', 'version', 'condition', 'round', 'addChoice']]
temp.head()

In [ ]:
v1 = temp[temp['version'] == 1]
v2 = temp[temp['version'] == 2]

df = v1.merge(v2, on = ['session_no', 'pid', 'round'], suffixes = ["_v1", "_v2"])

df = df[['session_no', 'pid', 'round', 'condition_v1', 'condition_v2', 'addChoice_v1', 'addChoice_v2']]
df.head()

In [ ]:
df['condition_pair']       = df.apply(get_condition_pairs,      axis = 1)
df['change_in_add_choice'] = df.apply(get_change_in_add_choice, axis = 1)

df = df[df['round'] < 9]
df.head()

In [ ]:
# y, X = patsy.dmatrices('change_in_add_choice ~ C(condition_pair) + C(round) - 1', df, return_type = 'dataframe')

mod = glm("change_in_add_choice ~ C(condition_pair) + C(round) - 1", data = df)

session_c = df[(df['change_in_add_choice'] == df['change_in_add_choice']) &
               (df['condition_pair']       == df['condition_pair'])        &
               (df['round']                == df['round'])]['session_no']

pid_c     = df[(df['change_in_add_choice'] == df['change_in_add_choice']) &
               (df['condition_pair']       == df['condition_pair'])        &
               (df['round']                == df['round'])]['pid']

# ols = sm.OLS(y, X)
# ols.fit(cov_type = 'cluster', cov_kwds = {'groups': [session_c, pid_c]}).summary()

mod.fit_constrained(
    'C(condition_pair)[RU to RU] + C(condition_pair)[RU to EE]  + \
     C(condition_pair)[RU to RE] + C(condition_pair)[RU to EU]  + \
     C(condition_pair)[EE to RU] + C(condition_pair)[EE to EE]  + \
     C(condition_pair)[EE to RE] + C(condition_pair)[EE to EU]  + \
     C(condition_pair)[RE to RU] + C(condition_pair)[RE to EE] + \
     C(condition_pair)[RE to RE] + C(condition_pair)[RE to EU] + \
     C(condition_pair)[EU to RU] + C(condition_pair)[EU to EE] + \
     C(condition_pair)[EU to RE] + C(condition_pair)[EU to EU]',
    cov_type = 'cluster', cov_kwds = {'groups': [session_c, pid_c]}).summary()

In [ ]:
#                   RU       EE       RE       EU
array = np.array([[-1.69, -4.15,  7.39,  3.10],  # RU
                  [ 0.49,  1.27,  2.07, -10.26], # EE
                  [-0.10, -0.57,  1.15, -1.91],  # RE
                  [ 6.39,  6.85, -1.90, -8.12]]) # EU

# annot = np.array([["",        "", "p=0.052", ""],
#                   ["",        "", "",        "p=0.005"],
#                   ["",        "", "",        ""],
#                   ["p=0.016", "", "",        "p=0.002"]])

annot = np.array([["", "", "", ""],
                  ["", "", "", ""],
                  ["", "", "", ""],
                  ["", "", "", ""]])

fig, ax = plt.subplots(figsize = (6,8))

ax = sns.heatmap(array, 
                 vmin = -11, 
                 vmax =  11, 
                 annot = annot, 
                 fmt = '', 
                 cbar_kws={'label': 'Change in add tie choice', 
                           'orientation': 'horizontal'},
                 cmap = "RdBu_r",)
ax.set_xticklabels(["RU", "EE", "RE", "EU"])
ax.set_yticklabels(["RU", "EE", "RE", "EU"], rotation = 0)
ax.set_xlabel("Second game")
ax.set_ylabel("First game")

fig.savefig(os.path.join(path_to_diss, "paper/figures/appendices/figureA6.3.png"), 
            bbox_inches = 'tight', 
            pad_inches = 0.25)

##### Between-individual models

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal + C(round)',
    data_final[(data_final['version'] == 1) & (data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']   == 1)                       &
                       (data_final['version']   == data_final['version'])   &
                       (data_final['addChoice'] == data_final['addChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 9)]['session_no']

pid_c     = data_final[(data_final['version']   == 1)                       &
                       (data_final['version']   == data_final['version'])   &
                       (data_final['addChoice'] == data_final['addChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 9)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal + coopChoiceAlter_add + C(round)', 
    data_final[(data_final['version'] == 1) & (data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']             == 1)                                 &
                       (data_final['version']             == data_final['version'])             &
                       (data_final['addChoice']           == data_final['addChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_add'] == data_final['coopChoiceAlter_add']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 9)]['session_no']

pid_c     = data_final[(data_final['version']             == 1)                                 &
                       (data_final['version']             == data_final['version'])             &
                       (data_final['addChoice']           == data_final['addChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_add'] == data_final['coopChoiceAlter_add']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 9)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal + C(round)', 
    data_final[(data_final['version'] == 2) & (data_final['round'] < 10)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']   == 2)                       &
                       (data_final['version']   == data_final['version'])   &
                       (data_final['addChoice'] == data_final['addChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 10)]['session_no']

pid_c     = data_final[(data_final['version']   == 2)                       &
                       (data_final['version']   == data_final['version'])   &
                       (data_final['addChoice'] == data_final['addChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 10)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal + coopChoiceAlter_add + C(round)', 
    data_final[(data_final['version'] == 2) & (data_final['round'] < 10)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']             == 2)                                 &
                       (data_final['version']             == data_final['version'])             &
                       (data_final['addChoice']           == data_final['addChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_add'] == data_final['coopChoiceAlter_add']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 10)]['session_no']

pid_c     = data_final[(data_final['version']             == 2)                                 &
                       (data_final['version']             == data_final['version'])             &
                       (data_final['addChoice']           == data_final['addChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_add'] == data_final['coopChoiceAlter_add']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 10)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal + C(version) + C(round)', 
    data_final[(data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']   == data_final['version'])   &
                       (data_final['addChoice'] == data_final['addChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 9)]['session_no']

pid_c     = data_final[(data_final['version']   == data_final['version'])   &
                       (data_final['addChoice'] == data_final['addChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 9)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal + C(version) + coopChoiceAlter_add + C(round)', 
    data_final[(data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']             == data_final['version'])             &
                       (data_final['addChoice']           == data_final['addChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_add'] == data_final['coopChoiceAlter_add']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 9)]['session_no']

pid_c     = data_final[(data_final['version']             == data_final['version'])             &
                       (data_final['addChoice']           == data_final['addChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_add'] == data_final['coopChoiceAlter_add']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 9)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal * C(version) + C(round)', 
    data_final[(data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']   == data_final['version'])   &
                       (data_final['addChoice'] == data_final['addChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 9)]['session_no']

pid_c     = data_final[(data_final['version']   == data_final['version'])   &
                       (data_final['addChoice'] == data_final['addChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 9)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal * C(version) + coopChoiceAlter_add + C(round)', 
    data_final[(data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']             == data_final['version'])             &
                       (data_final['addChoice']           == data_final['addChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_add'] == data_final['coopChoiceAlter_add']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 9)]['session_no']

pid_c     = data_final[(data_final['version']             == data_final['version'])             &
                       (data_final['addChoice']           == data_final['addChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_add'] == data_final['coopChoiceAlter_add']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 9)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Between-session models

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal + C(round)', 
    session_data[(session_data['version'] == 1) & (session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']   == 1)                         &
                         (session_data['version']   == session_data['version'])   &
                         (session_data['addChoice'] == session_data['addChoice']) &
                         (session_data['earned']    == session_data['earned'])    &
                         (session_data['equal']     == session_data['equal'])     &
                         (session_data['round']     == session_data['round'])     &
                         (session_data['round'] < 9)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal + coopChoiceAlter_add + C(round)', 
    session_data[(session_data['version'] == 1) & (session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']             == 1)                                   &
                         (session_data['version']             == session_data['version'])             &
                         (session_data['addChoice']           == session_data['addChoice'])           &
                         (session_data['earned']              == session_data['earned'])              &
                         (session_data['equal']               == session_data['equal'])               &
                         (session_data['coopChoiceAlter_add'] == session_data['coopChoiceAlter_add']) &
                         (session_data['round']               == session_data['round'])               &
                         (session_data['round'] < 9)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal + C(round)', 
    session_data[(session_data['version'] == 2) & (session_data['round'] < 10)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']   == 2)                         &
                         (session_data['version']   == session_data['version'])   &
                         (session_data['addChoice'] == session_data['addChoice']) &
                         (session_data['earned']    == session_data['earned'])    &
                         (session_data['equal']     == session_data['equal'])     &
                         (session_data['round']     == session_data['round'])     &
                         (session_data['round'] < 10)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal + coopChoiceAlter_add + C(round)', 
    session_data[(session_data['version'] == 2) & (session_data['round'] < 10)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']             == 2)                                   &
                         (session_data['version']             == session_data['version'])             &
                         (session_data['addChoice']           == session_data['addChoice'])           &
                         (session_data['earned']              == session_data['earned'])              &
                         (session_data['equal']               == session_data['equal'])               &
                         (session_data['coopChoiceAlter_add'] == session_data['coopChoiceAlter_add']) &
                         (session_data['round']               == session_data['round'])               &
                         (session_data['round'] < 10)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal + C(version) + C(round)', 
    session_data[(session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']   == session_data['version'])   &
                         (session_data['addChoice'] == session_data['addChoice']) &
                         (session_data['earned']    == session_data['earned'])    &
                         (session_data['equal']     == session_data['equal'])     &
                         (session_data['round']     == session_data['round'])     &
                         (session_data['round'] < 9)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal + C(version) + coopChoiceAlter_add + C(round)', 
    session_data[(session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']             == session_data['version'])             &
                         (session_data['addChoice']           == session_data['addChoice'])           &
                         (session_data['earned']              == session_data['earned'])              &
                         (session_data['equal']               == session_data['equal'])               &
                         (session_data['coopChoiceAlter_add'] == session_data['coopChoiceAlter_add']) &
                         (session_data['round']               == session_data['round'])               &
                         (session_data['round'] < 9)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal * C(version) + C(round)', 
    session_data[(session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']   == session_data['version'])   &
                         (session_data['addChoice'] == session_data['addChoice']) &
                         (session_data['earned']    == session_data['earned'])    &
                         (session_data['equal']     == session_data['equal'])     &
                         (session_data['round']     == session_data['round'])     &
                         (session_data['round'] < 9)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ earned * equal * C(version) + coopChoiceAlter_add + C(round)', 
    session_data[(session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']             == session_data['version'])             &
                         (session_data['addChoice']           == session_data['addChoice'])           &
                         (session_data['earned']              == session_data['earned'])              &
                         (session_data['equal']               == session_data['equal'])               &
                         (session_data['coopChoiceAlter_add'] == session_data['coopChoiceAlter_add']) &
                         (session_data['round']               == session_data['round'])               &
                         (session_data['round'] < 9)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

##### Within-individual models

In [ ]:
y, X = patsy.dmatrices(
    'delta_addChoice ~ delta_earned + delta_equal + delta_int + C(round)', 
    # 'delta_addChoice ~ delta_earned * delta_equal + C(round)'
    data_within[(data_within['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_within[(data_within['round']           == data_within['round'])           &
                        (data_within['delta_addChoice'] == data_within['delta_addChoice']) &
                        (data_within['delta_earned']    == data_within['delta_earned'])    &
                        (data_within['delta_equal']     == data_within['delta_equal'])     &
                        (data_within['round'] < 9)]['session_no']

pid_c     = data_within[(data_within['round']           == data_within['round'])           &
                        (data_within['delta_addChoice'] == data_within['delta_addChoice']) &
                        (data_within['delta_earned']    == data_within['delta_earned'])    &
                        (data_within['delta_equal']     == data_within['delta_equal'])     &
                        (data_within['round'] < 9)]['pid']

print(len(np.unique(pid_c)))

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'delta_addChoice ~ delta_earned + delta_equal + delta_int + delta_coopChoiceAlter_add + C(round)', 
    # 'delta_addChoice ~ delta_earned * delta_equal + delta_coopChoiceAlter_add + C(round)',
    data_within[(data_within['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_within[
    (data_within['round']                     == data_within['round'])                     &
    (data_within['delta_addChoice']           == data_within['delta_addChoice'])           &
    (data_within['delta_earned']              == data_within['delta_earned'])              &
    (data_within['delta_equal']               == data_within['delta_equal'])               &
    (data_within['delta_coopChoiceAlter_add'] == data_within['delta_coopChoiceAlter_add']) &
    (data_within['round'] < 9)
]['session_no']

pid_c = data_within[
    (data_within['round']                     == data_within['round'])                     &
    (data_within['delta_addChoice']           == data_within['delta_addChoice'])           &
    (data_within['delta_earned']              == data_within['delta_earned'])              &
    (data_within['delta_equal']               == data_within['delta_equal'])               &
    (data_within['delta_coopChoiceAlter_add'] == data_within['delta_coopChoiceAlter_add']) &
    (data_within['round'] < 9)
]['pid']

print(len(np.unique(pid_c)))

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Within-session models

In [ ]:
y, X = patsy.dmatrices(
    'delta_addChoice ~ delta_earned + delta_equal + delta_int + C(round)',
    session_data_within[(session_data_within['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data_within[
    (session_data_within['round']           == session_data_within['round'])           &
    (session_data_within['delta_addChoice'] == session_data_within['delta_addChoice']) &
    (session_data_within['delta_earned']    == session_data_within['delta_earned'])    &
    (session_data_within['delta_equal']     == session_data_within['delta_equal'])     &
    (session_data_within['round'] < 9)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'delta_addChoice ~ delta_earned + delta_equal + delta_int + delta_coopChoiceAlter_add + C(round)',
    session_data_within[(session_data_within['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data_within[
    (session_data_within['round']                     == session_data_within['round'])                     &
    (session_data_within['delta_addChoice']           == session_data_within['delta_addChoice'])           &
    (session_data_within['delta_earned']              == session_data_within['delta_earned'])              &
    (session_data_within['delta_equal']               == session_data_within['delta_equal'])               &
    (session_data_within['delta_coopChoiceAlter_add'] == session_data_within['delta_coopChoiceAlter_add']) &
    (session_data_within['round'] < 9)
]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

### Tie breakage

##### Overview

In [ ]:
print(np.mean(data_final.cutChoice))
print()

print(data_final.groupby('version')['cutChoice'].mean())
print()

print(data_final.groupby('round')['cutChoice'].mean())
print()

In [ ]:
print(data_final.groupby('condition')['cutChoice'].mean())
print()

print(data_final.groupby(['version', 'condition'])['cutChoice'].mean())

##### Figures

In [ ]:
data_final.groupby(['round', 'version']).mean()['cutChoice'].unstack().plot(style = ['-', '--'], color = 'black')
plt.xlabel('Round')
plt.ylabel('Average Cut Tie Choice')
plt.title('Average Cut Tie Choice by Round')
plt.legend(['1st  PG game', '2nd PG game'])
plt.xticks(np.arange(1, 10, step=1))

plt.savefig(os.path.join(path_to_diss, "paper/figures/appendices/figureA6.4.png"))

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (15, 5))
clrs = ['royalblue', 'orange', 'limegreen', 'tomato']

for i in range(3):
    if i == 0:
        temp = data_final[
            data_final['version'] == 1].groupby(['round',
                                                 'condition']).mean()['cutChoice'].unstack().plot(color = clrs, 
                                                                                                  ax = ax[i])
        ax[i].set_title('First game only')
        ax[i].set_xticks(np.arange(1, 9, step = 1))
    elif i == 1:
        temp = data_final[
            data_final['version'] == 2].groupby(['round',
                                                 'condition']).mean()['cutChoice'].unstack().plot(color = clrs,
                                                                                                  ax = ax[i])
        ax[i].set_title('Second game only')
        ax[i].set_xticks(np.arange(1, 10, step = 1))
    else:
        temp = data_final[
            data_final['round'] < 9].groupby(['round',
                                              'condition']).mean()['cutChoice'].unstack().plot(color = clrs,
                                                                                               ax = ax[i])
        ax[i].set_title('Pooled')
        ax[i].set_xticks(np.arange(1, 9, step = 1))
    ax[i].set_xlabel('Round')
    ax[i].set_ylabel('Average Cut Tie Choice')
    ax[i].set_yticks(np.arange(0.20, 0.55, step = 0.05))
    ax[i].legend(['EE', 'EU', 'RE', 'RU'], loc = 'upper left')
    ax[i].set(ylim=(0.20, 0.50))
    
fig.savefig(os.path.join(path_to_diss, "paper/figures/appendices/figureA6.5.png"), 
            bbox_inches = 'tight', 
            pad_inches = 0.25)

In [ ]:
def get_condition_pairs(row):
    if   row.condition_v1 == 'ru' and row.condition_v2 == 'ru': return 'RU to RU'
    elif row.condition_v1 == 'ru' and row.condition_v2 == 'ee': return 'RU to EE'
    elif row.condition_v1 == 'ru' and row.condition_v2 == 're': return 'RU to RE'
    elif row.condition_v1 == 'ru' and row.condition_v2 == 'eu': return 'RU to EU'
    elif row.condition_v1 == 'ee' and row.condition_v2 == 'ru': return 'EE to RU'
    elif row.condition_v1 == 'ee' and row.condition_v2 == 'ee': return 'EE to EE'
    elif row.condition_v1 == 'ee' and row.condition_v2 == 're': return 'EE to RE'
    elif row.condition_v1 == 'ee' and row.condition_v2 == 'eu': return 'EE to EU'
    elif row.condition_v1 == 're' and row.condition_v2 == 'ru': return 'RE to RU'
    elif row.condition_v1 == 're' and row.condition_v2 == 'ee': return 'RE to EE'
    elif row.condition_v1 == 're' and row.condition_v2 == 're': return 'RE to RE'
    elif row.condition_v1 == 're' and row.condition_v2 == 'eu': return 'RE to EU'
    elif row.condition_v1 == 'eu' and row.condition_v2 == 'ru': return 'EU to RU'
    elif row.condition_v1 == 'eu' and row.condition_v2 == 'ee': return 'EU to EE'
    elif row.condition_v1 == 'eu' and row.condition_v2 == 're': return 'EU to RE'
    elif row.condition_v1 == 'eu' and row.condition_v2 == 'eu': return 'EU to EU'
    
def get_change_in_cut_choice(row):
    return row.cutChoice_v2 - row.cutChoice_v1

In [ ]:
temp = data_final[['session_no', 'pid', 'version', 'condition', 'round', 'cutChoice']]
temp.head()

In [ ]:
v1 = temp[temp['version'] == 1]
v2 = temp[temp['version'] == 2]

df = v1.merge(v2, on = ['session_no', 'pid', 'round'], suffixes = ["_v1", "_v2"])

df = df[['session_no', 'pid', 'round', 'condition_v1', 'condition_v2', 'cutChoice_v1', 'cutChoice_v2']]
df.head()

In [ ]:
df['condition_pair']       = df.apply(get_condition_pairs,      axis = 1)
df['change_in_cut_choice'] = df.apply(get_change_in_cut_choice, axis = 1)

df = df[df['round'] < 9]
df.head()

In [ ]:
# y, X = patsy.dmatrices('change_in_cut_choice ~ C(condition_pair) + C(round) - 1', df, return_type = 'dataframe')

mod = glm("change_in_cut_choice ~ C(condition_pair) + C(round) - 1", data = df)

session_c = df[(df['change_in_cut_choice'] == df['change_in_cut_choice']) &
               (df['condition_pair']       == df['condition_pair'])        &
               (df['round']                == df['round'])]['session_no']

pid_c     = df[(df['change_in_cut_choice'] == df['change_in_cut_choice']) &
               (df['condition_pair']       == df['condition_pair'])        &
               (df['round']                == df['round'])]['pid']

# ols = sm.OLS(y, X)
# ols.fit(cov_type = 'cluster', cov_kwds = {'groups': [session_c, pid_c]}).summary()

mod.fit_constrained(
    'C(condition_pair)[RU to RU] + C(condition_pair)[RU to EE]  + \
     C(condition_pair)[RU to RE] + C(condition_pair)[RU to EU]  + \
     C(condition_pair)[EE to RU] + C(condition_pair)[EE to EE]  + \
     C(condition_pair)[EE to RE] + C(condition_pair)[EE to EU]  + \
     C(condition_pair)[RE to RU] + C(condition_pair)[RE to EE] + \
     C(condition_pair)[RE to RE] + C(condition_pair)[RE to EU] + \
     C(condition_pair)[EU to RU] + C(condition_pair)[EU to EE] + \
     C(condition_pair)[EU to RE] + C(condition_pair)[EU to EU]',
    cov_type = 'cluster', cov_kwds = {'groups': [session_c, pid_c]}).summary()

In [ ]:
#                   RU       EE       RE       EU
array = np.array([[-2.67,  0.04, -5.20,  1.58],  # RU
                  [ 0.65,  3.90, -12.86, 6.13],  # EE
                  [ 5.72,  1.81, -5.05,  2.85],  # RE
                  [ 0.49, -6.41,  4.27,  4.75]]) # EU

# annot = np.array([["", "", "",        ""],
#                   ["", "", "p=0.049", ""],
#                   ["", "", "",        ""],
#                   ["", "", "",        ""]])

annot = np.array([["", "", "", ""],
                  ["", "", "", ""],
                  ["", "", "", ""],
                  ["", "", "", ""]])

fig, ax = plt.subplots(figsize = (6,8))

ax = sns.heatmap(array, 
                 vmin = -13, 
                 vmax =  13, 
                 annot = annot, 
                 fmt = '', 
                 cbar_kws={'label': 'Change in cut tie choice', 
                           'orientation': 'horizontal'},
                 cmap = "RdBu_r",)
ax.set_xticklabels(["RU", "EE", "RE", "EU"])
ax.set_yticklabels(["RU", "EE", "RE", "EU"], rotation = 0)
ax.set_xlabel("Second game")
ax.set_ylabel("First game")

fig.savefig(os.path.join(path_to_diss, "paper/figures/appendices/figureA6.6.png"), 
            bbox_inches = 'tight', 
            pad_inches = 0.25)

##### Between-individual models

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal + C(round)',
    data_final[(data_final['version'] == 1) & (data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']   == 1)                       &
                       (data_final['version']   == data_final['version'])   &
                       (data_final['cutChoice'] == data_final['cutChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 9)]['session_no']

pid_c     = data_final[(data_final['version']   == 1)                       &
                       (data_final['version']   == data_final['version'])   &
                       (data_final['cutChoice'] == data_final['cutChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 9)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal + coopChoiceAlter_cut + C(round)', 
    data_final[(data_final['version'] == 1) & (data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']             == 1)                                 &
                       (data_final['version']             == data_final['version'])             &
                       (data_final['cutChoice']           == data_final['cutChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_cut'] == data_final['coopChoiceAlter_cut']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 9)]['session_no']

pid_c     = data_final[(data_final['version']             == 1)                                 &
                       (data_final['version']             == data_final['version'])             &
                       (data_final['cutChoice']           == data_final['cutChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_cut'] == data_final['coopChoiceAlter_cut']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 9)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal + C(round)', 
    data_final[(data_final['version'] == 2) & (data_final['round'] < 10)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']   == 2)                       &
                       (data_final['version']   == data_final['version'])   &
                       (data_final['cutChoice'] == data_final['cutChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 10)]['session_no']

pid_c     = data_final[(data_final['version']   == 2)                       &
                       (data_final['version']   == data_final['version'])   &
                       (data_final['cutChoice'] == data_final['cutChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 10)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal + coopChoiceAlter_cut + C(round)', 
    data_final[(data_final['version'] == 2) & (data_final['round'] < 10)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']             == 2)                                 &
                       (data_final['version']             == data_final['version'])             &
                       (data_final['cutChoice']           == data_final['cutChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_cut'] == data_final['coopChoiceAlter_cut']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 10)]['session_no']

pid_c     = data_final[(data_final['version']             == 2)                                 &
                       (data_final['version']             == data_final['version'])             &
                       (data_final['cutChoice']           == data_final['cutChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_cut'] == data_final['coopChoiceAlter_cut']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 10)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal + C(version) + C(round)', 
    data_final[(data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']   == data_final['version'])   &
                       (data_final['cutChoice'] == data_final['cutChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 9)]['session_no']

pid_c     = data_final[(data_final['version']   == data_final['version'])   &
                       (data_final['cutChoice'] == data_final['cutChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 9)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal + C(version) + coopChoiceAlter_cut + C(round)', 
    data_final[(data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']             == data_final['version'])             &
                       (data_final['cutChoice']           == data_final['cutChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_cut'] == data_final['coopChoiceAlter_cut']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 9)]['session_no']

pid_c     = data_final[(data_final['version']             == data_final['version'])             &
                       (data_final['cutChoice']           == data_final['cutChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_cut'] == data_final['coopChoiceAlter_cut']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 9)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal * C(version) + C(round)', 
    data_final[(data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']   == data_final['version'])   &
                       (data_final['cutChoice'] == data_final['cutChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 9)]['session_no']

pid_c     = data_final[(data_final['version']   == data_final['version'])   &
                       (data_final['cutChoice'] == data_final['cutChoice']) &
                       (data_final['earned']    == data_final['earned'])    &
                       (data_final['equal']     == data_final['equal'])     &
                       (data_final['round']     == data_final['round'])     &
                       (data_final['round'] < 9)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal * C(version) + coopChoiceAlter_cut + C(round)', 
    data_final[(data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(data_final['version']             == data_final['version'])             &
                       (data_final['cutChoice']           == data_final['cutChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_cut'] == data_final['coopChoiceAlter_cut']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 9)]['session_no']

pid_c     = data_final[(data_final['version']             == data_final['version'])             &
                       (data_final['cutChoice']           == data_final['cutChoice'])           &
                       (data_final['earned']              == data_final['earned'])              &
                       (data_final['equal']               == data_final['equal'])               &
                       (data_final['coopChoiceAlter_cut'] == data_final['coopChoiceAlter_cut']) &
                       (data_final['round']               == data_final['round'])               &
                       (data_final['round'] < 9)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Between-session models

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal + C(round)', 
    session_data[(session_data['version'] == 1) & (session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']   == 1)                         &
                         (session_data['version']   == session_data['version'])   &
                         (session_data['cutChoice'] == session_data['cutChoice']) &
                         (session_data['earned']    == session_data['earned'])    &
                         (session_data['equal']     == session_data['equal'])     &
                         (session_data['round']     == session_data['round'])     &
                         (session_data['round'] < 9)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal + coopChoiceAlter_cut + C(round)', 
    session_data[(session_data['version'] == 1) & (session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']             == 1)                                   &
                         (session_data['version']             == session_data['version'])             &
                         (session_data['cutChoice']           == session_data['cutChoice'])           &
                         (session_data['earned']              == session_data['earned'])              &
                         (session_data['equal']               == session_data['equal'])               &
                         (session_data['coopChoiceAlter_cut'] == session_data['coopChoiceAlter_cut']) &
                         (session_data['round']               == session_data['round'])               &
                         (session_data['round'] < 9)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal + C(round)', 
    session_data[(session_data['version'] == 2) & (session_data['round'] < 10)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']   == 2)                         &
                         (session_data['version']   == session_data['version'])   &
                         (session_data['cutChoice'] == session_data['cutChoice']) &
                         (session_data['earned']    == session_data['earned'])    &
                         (session_data['equal']     == session_data['equal'])     &
                         (session_data['round']     == session_data['round'])     &
                         (session_data['round'] < 10)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal + coopChoiceAlter_cut + C(round)', 
    session_data[(session_data['version'] == 2) & (session_data['round'] < 10)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']             == 2)                                   &
                         (session_data['version']             == session_data['version'])             &
                         (session_data['cutChoice']           == session_data['cutChoice'])           &
                         (session_data['earned']              == session_data['earned'])              &
                         (session_data['equal']               == session_data['equal'])               &
                         (session_data['coopChoiceAlter_cut'] == session_data['coopChoiceAlter_cut']) &
                         (session_data['round']               == session_data['round'])               &
                         (session_data['round'] < 10)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal + C(version) + C(round)', 
    session_data[(session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']   == session_data['version'])   &
                         (session_data['cutChoice'] == session_data['cutChoice']) &
                         (session_data['earned']    == session_data['earned'])    &
                         (session_data['equal']     == session_data['equal'])     &
                         (session_data['round']     == session_data['round'])     &
                         (session_data['round'] < 9)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal + C(version) + coopChoiceAlter_cut + C(round)', 
    session_data[(session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']             == session_data['version'])             &
                         (session_data['cutChoice']           == session_data['cutChoice'])           &
                         (session_data['earned']              == session_data['earned'])              &
                         (session_data['equal']               == session_data['equal'])               &
                         (session_data['coopChoiceAlter_cut'] == session_data['coopChoiceAlter_cut']) &
                         (session_data['round']               == session_data['round'])               &
                         (session_data['round'] < 9)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal * C(version) + C(round)', 
    session_data[(session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']   == session_data['version'])   &
                         (session_data['cutChoice'] == session_data['cutChoice']) &
                         (session_data['earned']    == session_data['earned'])    &
                         (session_data['equal']     == session_data['equal'])     &
                         (session_data['round']     == session_data['round'])     &
                         (session_data['round'] < 9)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ earned * equal * C(version) + coopChoiceAlter_cut + C(round)', 
    session_data[(session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(session_data['version']             == session_data['version'])             &
                         (session_data['cutChoice']           == session_data['cutChoice'])           &
                         (session_data['earned']              == session_data['earned'])              &
                         (session_data['equal']               == session_data['equal'])               &
                         (session_data['coopChoiceAlter_cut'] == session_data['coopChoiceAlter_cut']) &
                         (session_data['round']               == session_data['round'])               &
                         (session_data['round'] < 9)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

##### Within-individual models

In [ ]:
y, X = patsy.dmatrices(
    'delta_cutChoice ~ delta_earned + delta_equal + delta_int + C(round)', 
    # 'delta_cutChoice ~ delta_earned * delta_equal + C(round)'
    data_within[(data_within['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_within[(data_within['round']           == data_within['round'])           &
                        (data_within['delta_cutChoice'] == data_within['delta_cutChoice']) &
                        (data_within['delta_earned']    == data_within['delta_earned'])    &
                        (data_within['delta_equal']     == data_within['delta_equal'])     &
                        (data_within['round'] < 9)]['session_no']

pid_c     = data_within[(data_within['round']           == data_within['round'])           &
                        (data_within['delta_cutChoice'] == data_within['delta_cutChoice']) &
                        (data_within['delta_earned']    == data_within['delta_earned'])    &
                        (data_within['delta_equal']     == data_within['delta_equal'])     &
                        (data_within['round'] < 9)]['pid']

print(len(np.unique(pid_c)))

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'delta_cutChoice ~ delta_earned + delta_equal + delta_int + delta_coopChoiceAlter_cut + C(round)', 
    # 'delta_cutChoice ~ delta_earned * delta_equal + delta_coopChoiceAlter_cut + C(round)', 
    data_within[(data_within['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_within[
    (data_within['round']                     == data_within['round'])                     &
    (data_within['delta_cutChoice']           == data_within['delta_cutChoice'])           &
    (data_within['delta_earned']              == data_within['delta_earned'])              &
    (data_within['delta_equal']               == data_within['delta_equal'])               &
    (data_within['delta_coopChoiceAlter_cut'] == data_within['delta_coopChoiceAlter_cut']) &
    (data_within['round'] < 9)
]['session_no']

pid_c = data_within[
    (data_within['round']                     == data_within['round'])                     &
    (data_within['delta_cutChoice']           == data_within['delta_cutChoice'])           &
    (data_within['delta_earned']              == data_within['delta_earned'])              &
    (data_within['delta_equal']               == data_within['delta_equal'])               &
    (data_within['delta_coopChoiceAlter_cut'] == data_within['delta_coopChoiceAlter_cut']) &
    (data_within['round'] < 9)
]['pid']

print(len(np.unique(pid_c)))

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Within-session models

In [ ]:
y, X = patsy.dmatrices(
    'delta_cutChoice ~ delta_earned + delta_equal + delta_int + C(round)',
    session_data_within[(session_data_within['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data_within[
    (session_data_within['round']           == session_data_within['round'])           &
    (session_data_within['delta_cutChoice'] == session_data_within['delta_cutChoice']) &
    (session_data_within['delta_earned']    == session_data_within['delta_earned'])    &
    (session_data_within['delta_equal']     == session_data_within['delta_equal'])     &
    (session_data_within['round'] < 9)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'delta_cutChoice ~ delta_earned + delta_equal + delta_int + delta_coopChoiceAlter_cut + C(round)',
    session_data_within[(session_data_within['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data_within[
    (session_data_within['round']                     == session_data_within['round'])                     &
    (session_data_within['delta_cutChoice']           == session_data_within['delta_cutChoice'])           &
    (session_data_within['delta_earned']              == session_data_within['delta_earned'])              &
    (session_data_within['delta_equal']               == session_data_within['delta_equal'])               &
    (session_data_within['delta_coopChoiceAlter_cut'] == session_data_within['delta_coopChoiceAlter_cut']) &
    (session_data_within['round'] < 9)
]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

### Local inequality

##### Cooperation ~ local inequality: between-individual

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ giniLocal + C(round)',
    data_final_coop[(~np.isinf(data_final_coop['giniLocal'])) & (data_final_coop['version'] == 1)],
    return_type = 'dataframe'
)

session_c = data_final_coop[(~np.isinf(data_final_coop['giniLocal']))                        & 
                            (data_final_coop['version']    == 1)                             & 
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) &
                            (data_final_coop['giniLocal']  == data_final_coop['giniLocal'])]['session_no']

pid_c     = data_final_coop[(~np.isinf(data_final_coop['giniLocal']))                        &
                            (data_final_coop['version']    == 1)                             & 
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) & 
                            (data_final_coop['giniLocal']  == data_final_coop['giniLocal'])]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ giniLocal + C(round)',
    data_final_coop[(~np.isinf(data_final_coop['giniLocal'])) & (data_final_coop['version'] == 2)],
    return_type = 'dataframe'
)

session_c = data_final_coop[(~np.isinf(data_final_coop['giniLocal']))                        & 
                            (data_final_coop['version']    == 2)                             & 
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) &
                            (data_final_coop['giniLocal']  == data_final_coop['giniLocal'])]['session_no']

pid_c     = data_final_coop[(~np.isinf(data_final_coop['giniLocal']))                        &
                            (data_final_coop['version']    == 2)                             & 
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) & 
                            (data_final_coop['giniLocal']  == data_final_coop['giniLocal'])]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ giniLocal + C(version) + C(round)',
    data_final_coop[(~np.isinf(data_final_coop['giniLocal'])) & (data_final_coop['round'] != 10)],
    return_type = 'dataframe'
)

session_c = data_final_coop[(~np.isinf(data_final_coop['giniLocal']))                        & 
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) &
                            (data_final_coop['giniLocal']  == data_final_coop['giniLocal'])  &
                            (data_final_coop['round']      != 10)]['session_no']

pid_c     = data_final_coop[(~np.isinf(data_final_coop['giniLocal']))                        &
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) & 
                            (data_final_coop['giniLocal']  == data_final_coop['giniLocal'])  &
                            (data_final_coop['round']      != 10)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ giniLocal * C(version) + C(round)',
    data_final_coop[(~np.isinf(data_final_coop['giniLocal'])) & (data_final_coop['round'] != 10)],
    return_type = 'dataframe'
)

session_c = data_final_coop[(~np.isinf(data_final_coop['giniLocal']))                        & 
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) &
                            (data_final_coop['giniLocal']  == data_final_coop['giniLocal'])  &
                            (data_final_coop['round']      != 10)]['session_no']

pid_c     = data_final_coop[(~np.isinf(data_final_coop['giniLocal']))                        &
                            (data_final_coop['coopChoice'] == data_final_coop['coopChoice']) & 
                            (data_final_coop['giniLocal']  == data_final_coop['giniLocal'])  &
                            (data_final_coop['round']      != 10)]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Cooperation ~ local inequality: between-session

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ giniLocal + C(round)',
    session_data_coop[(~np.isinf(session_data_coop['giniLocal'])) & (session_data_coop['version'] == 1)],
    return_type = 'dataframe'
)

session_c = session_data_coop[(~np.isinf(session_data_coop['giniLocal']))                          & 
                              (session_data_coop['version']    == 1)                               & 
                              (session_data_coop['coopChoice'] == session_data_coop['coopChoice']) &
                              (session_data_coop['giniLocal']  == session_data_coop['giniLocal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ giniLocal + C(round)',
    session_data_coop[(~np.isinf(session_data_coop['giniLocal'])) & (session_data_coop['version'] == 2)],
    return_type = 'dataframe'
)

session_c = session_data_coop[(~np.isinf(session_data_coop['giniLocal']))                          & 
                              (session_data_coop['version']    == 2)                               & 
                              (session_data_coop['coopChoice'] == session_data_coop['coopChoice']) &
                              (session_data_coop['giniLocal']  == session_data_coop['giniLocal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ giniLocal + C(version) + C(round)',
    session_data_coop[(~np.isinf(session_data_coop['giniLocal'])) & (data_final_coop['round'] != 10)],
    return_type = 'dataframe'
)

session_c = session_data_coop[(~np.isinf(session_data_coop['giniLocal']))                          & 
                              (session_data_coop['coopChoice'] == session_data_coop['coopChoice']) &
                              (session_data_coop['giniLocal']  == session_data_coop['giniLocal'])  &
                              (data_final_coop['round']        != 10)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'coopChoice ~ giniLocal * C(version) + C(round)',
    session_data_coop[(~np.isinf(session_data_coop['giniLocal'])) & (data_final_coop['round'] != 10)],
    return_type = 'dataframe'
)

session_c = session_data_coop[(~np.isinf(session_data_coop['giniLocal']))                          & 
                              (session_data_coop['coopChoice'] == session_data_coop['coopChoice']) &
                              (session_data_coop['giniLocal']  == session_data_coop['giniLocal'])  &
                              (data_final_coop['round']        != 10)]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

##### Cooperation ~ local inequality: within-individual

In [ ]:
y, X = patsy.dmatrices(
    'delta_coopChoice ~ delta_giniLocal + C(round)',
    data_within_coop[(~np.isinf(data_within_coop['delta_giniLocal']))],
    return_type = 'dataframe'
)

session_c = data_within_coop[
    (~np.isinf(data_within_coop['delta_giniLocal']))                               & 
    (data_within_coop['delta_coopChoice'] == data_within_coop['delta_coopChoice']) &
    (data_within_coop['delta_giniLocal']  == data_within_coop['delta_giniLocal'])
]['session_no']

pid_c = data_within_coop[
    (~np.isinf(data_within_coop['delta_giniLocal']))                               & 
    (data_within_coop['delta_coopChoice'] == data_within_coop['delta_coopChoice']) &
    (data_within_coop['delta_giniLocal']  == data_within_coop['delta_giniLocal'])
]['pid']

print(len(np.unique(pid_c)))

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Cooperation ~ local inequality: within-session

In [ ]:
y, X = patsy.dmatrices(
    'delta_coopChoice ~ delta_giniLocal + C(round)',
    session_data_within_coop[(~np.isinf(session_data_within_coop['delta_giniLocal']))],
    return_type = 'dataframe'
)

session_c = session_data_within_coop[
    (~np.isinf(session_data_within_coop['delta_giniLocal']))                                       & 
    (session_data_within_coop['delta_coopChoice'] == session_data_within_coop['delta_coopChoice']) &
    (session_data_within_coop['delta_giniLocal']  == session_data_within_coop['delta_giniLocal'])
]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

##### Add choice ~ local inequality: between-individual

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ giniLocal + C(round)', 
    data_final[(~np.isinf(data_final['giniLocal'])) & (data_final['round'] < 9) & (data_final['version'] == 1)],
    return_type = 'dataframe'
)

session_c = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['version']   == 1)                       &
                       (data_final['round']     <  9)                       &
                       (data_final['addChoice'] == data_final['addChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['session_no']

pid_c     = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['version']   == 1)                       &
                       (data_final['round']     <  9)                       &
                       (data_final['addChoice'] == data_final['addChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ giniLocal + C(round)', 
    data_final[(~np.isinf(data_final['giniLocal'])) & (data_final['round'] < 10) & (data_final['version'] == 2)],
    return_type = 'dataframe'
)

session_c = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['version']   == 2)                       &
                       (data_final['round']     <  10)                      &
                       (data_final['addChoice'] == data_final['addChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['session_no']

pid_c     = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['version']   == 2)                       &
                       (data_final['round']     <  10)                      &
                       (data_final['addChoice'] == data_final['addChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ giniLocal + C(version) + C(round)', 
    data_final[(~np.isinf(data_final['giniLocal'])) & (data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['round']     <  9)                       &
                       (data_final['addChoice'] == data_final['addChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['session_no']

pid_c     = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['round']     <  9)                       &
                       (data_final['addChoice'] == data_final['addChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ giniLocal * C(version) + C(round)', 
    data_final[(~np.isinf(data_final['giniLocal'])) & (data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['round']     <  9)                       &
                       (data_final['addChoice'] == data_final['addChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['session_no']

pid_c     = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['round']     <  9)                       &
                       (data_final['addChoice'] == data_final['addChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Add choice ~ local inequality: between-session

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ giniLocal + C(round)', 
    session_data[(~np.isinf(session_data['giniLocal'])) & 
                 (session_data['round']   < 9)          & 
                 (session_data['version'] == 1)],
    return_type = 'dataframe'
)

session_c = session_data[(~np.isinf(session_data['giniLocal']))                   &
                         (session_data['version']   == 1)                         &
                         (session_data['round']     <  9)                         &
                         (session_data['addChoice'] == session_data['addChoice']) & 
                         (session_data['giniLocal'] == session_data['giniLocal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ giniLocal + C(round)', 
    session_data[(~np.isinf(session_data['giniLocal'])) & 
                 (session_data['round']   <  10)        & 
                 (session_data['version'] == 2)],
    return_type = 'dataframe'
)

session_c = session_data[(~np.isinf(session_data['giniLocal']))                   &
                         (session_data['version']   == 2)                         &
                         (session_data['round']     <  10)                        &
                         (session_data['addChoice'] == session_data['addChoice']) & 
                         (session_data['giniLocal'] == session_data['giniLocal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ giniLocal + C(version) + C(round)', 
    session_data[(~np.isinf(session_data['giniLocal'])) & (session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(~np.isinf(session_data['giniLocal']))                   &
                         (session_data['round']     <  9)                         &
                         (session_data['addChoice'] == session_data['addChoice']) & 
                         (session_data['giniLocal'] == session_data['giniLocal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'addChoice ~ giniLocal * C(version) + C(round)', 
    session_data[(~np.isinf(session_data['giniLocal'])) & (session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(~np.isinf(session_data['giniLocal']))                   &
                         (session_data['round']     <  9)                         &
                         (session_data['addChoice'] == session_data['addChoice']) & 
                         (session_data['giniLocal'] == session_data['giniLocal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

##### Add choice ~ local inequality: within-individual

In [ ]:
y, X = patsy.dmatrices(
    'delta_addChoice ~ delta_giniLocal + C(round)',
    data_within[(~np.isinf(data_within['delta_giniLocal'])) & (data_within['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_within[
    (~np.isinf(data_within['delta_giniLocal']))                        & 
    (data_within['delta_addChoice'] == data_within['delta_addChoice']) &
    (data_within['delta_giniLocal'] == data_within['delta_giniLocal']) & 
    (data_within['round'] < 9)
]['session_no']

pid_c = data_within[
    (~np.isinf(data_within['delta_giniLocal']))                        & 
    (data_within['delta_addChoice'] == data_within['delta_addChoice']) &
    (data_within['delta_giniLocal'] == data_within['delta_giniLocal']) &
    (data_within['round'] < 9)
]['pid']

print(len(np.unique(pid_c)))

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Add choice ~ local inequality: within-session

In [ ]:
y, X = patsy.dmatrices(
    'delta_addChoice ~ delta_giniLocal + C(round)',
    session_data_within[(~np.isinf(session_data_within['delta_giniLocal'])) & (session_data_within['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data_within[
    (~np.isinf(session_data_within['delta_giniLocal']))                                & 
    (session_data_within['delta_addChoice'] == session_data_within['delta_addChoice']) &
    (session_data_within['delta_giniLocal'] == session_data_within['delta_giniLocal']) & 
    (session_data_within['round'] < 9)
]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

##### Cut choice ~ local inequality: between-individual

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ giniLocal + C(round)', 
    data_final[(~np.isinf(data_final['giniLocal'])) & (data_final['round'] < 9) & (data_final['version'] == 1)],
    return_type = 'dataframe'
)

session_c = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['version']   == 1)                       &
                       (data_final['round']     <  9)                       &
                       (data_final['cutChoice'] == data_final['cutChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['session_no']

pid_c     = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['version']   == 1)                       &
                       (data_final['round']     <  9)                       &
                       (data_final['cutChoice'] == data_final['cutChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ giniLocal + C(round)', 
    data_final[(~np.isinf(data_final['giniLocal'])) & (data_final['round'] < 10) & (data_final['version'] == 2)],
    return_type = 'dataframe'
)

session_c = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['version']   == 2)                       &
                       (data_final['round']     <  10)                      &
                       (data_final['cutChoice'] == data_final['cutChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['session_no']

pid_c     = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['version']   == 2)                       &
                       (data_final['round']     <  10)                      &
                       (data_final['cutChoice'] == data_final['cutChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ giniLocal + C(version) + C(round)', 
    data_final[(~np.isinf(data_final['giniLocal'])) & (data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['round']     <  9)                       &
                       (data_final['cutChoice'] == data_final['cutChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['session_no']

pid_c     = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['round']     <  9)                       &
                       (data_final['cutChoice'] == data_final['cutChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ giniLocal * C(version) + C(round)', 
    data_final[(~np.isinf(data_final['giniLocal'])) & (data_final['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['round']     <  9)                       &
                       (data_final['cutChoice'] == data_final['cutChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['session_no']

pid_c     = data_final[(~np.isinf(data_final['giniLocal']))                 &
                       (data_final['round']     <  9)                       &
                       (data_final['cutChoice'] == data_final['cutChoice']) & 
                       (data_final['giniLocal'] == data_final['giniLocal'])]['pid']

print(len(np.unique(pid_c)))

logit = sm.Logit(y, X)
logit.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Cut choice ~ local inequality: between-session

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ giniLocal + C(round)', 
    session_data[(~np.isinf(session_data['giniLocal'])) & 
                 (session_data['round']   < 9)          & 
                 (session_data['version'] == 1)],
    return_type = 'dataframe'
)

session_c = session_data[(~np.isinf(session_data['giniLocal']))                   &
                         (session_data['version']   == 1)                         &
                         (session_data['round']     <  9)                         &
                         (session_data['cutChoice'] == session_data['cutChoice']) & 
                         (session_data['giniLocal'] == session_data['giniLocal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ giniLocal + C(round)', 
    session_data[(~np.isinf(session_data['giniLocal'])) & 
                 (session_data['round']   <  10)        & 
                 (session_data['version'] == 2)],
    return_type = 'dataframe'
)

session_c = session_data[(~np.isinf(session_data['giniLocal']))                   &
                         (session_data['version']   == 2)                         &
                         (session_data['round']     <  10)                        &
                         (session_data['cutChoice'] == session_data['cutChoice']) & 
                         (session_data['giniLocal'] == session_data['giniLocal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ giniLocal + C(version) + C(round)', 
    session_data[(~np.isinf(session_data['giniLocal'])) & (session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(~np.isinf(session_data['giniLocal']))                   &
                         (session_data['round']     <  9)                         &
                         (session_data['cutChoice'] == session_data['cutChoice']) & 
                         (session_data['giniLocal'] == session_data['giniLocal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

In [ ]:
y, X = patsy.dmatrices(
    'cutChoice ~ giniLocal * C(version) + C(round)', 
    session_data[(~np.isinf(session_data['giniLocal'])) & (session_data['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data[(~np.isinf(session_data['giniLocal']))                   &
                         (session_data['round']     <  9)                         &
                         (session_data['cutChoice'] == session_data['cutChoice']) & 
                         (session_data['giniLocal'] == session_data['giniLocal'])]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

##### Cut choice ~ local inequality: within-individual

In [ ]:
y, X = patsy.dmatrices(
    'delta_cutChoice ~ delta_giniLocal + C(round)',
    data_within[(~np.isinf(data_within['delta_giniLocal'])) & (data_within['round'] < 9)],
    return_type = 'dataframe'
)

session_c = data_within[
    (~np.isinf(data_within['delta_giniLocal']))                        & 
    (data_within['delta_cutChoice'] == data_within['delta_cutChoice']) &
    (data_within['delta_giniLocal'] == data_within['delta_giniLocal']) & 
    (data_within['round'] < 9)
]['session_no']

pid_c = data_within[
    (~np.isinf(data_within['delta_giniLocal']))                        & 
    (data_within['delta_cutChoice'] == data_within['delta_cutChoice']) &
    (data_within['delta_giniLocal'] == data_within['delta_giniLocal']) &
    (data_within['round'] < 9)
]['pid']

print(len(np.unique(pid_c)))

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c, pid_c]}).summary()

##### Add choice ~ local inequality: within-session

In [ ]:
y, X = patsy.dmatrices(
    'delta_cutChoice ~ delta_giniLocal + C(round)',
    session_data_within[(~np.isinf(session_data_within['delta_giniLocal'])) & (session_data_within['round'] < 9)],
    return_type = 'dataframe'
)

session_c = session_data_within[
    (~np.isinf(session_data_within['delta_giniLocal']))                                & 
    (session_data_within['delta_cutChoice'] == session_data_within['delta_cutChoice']) &
    (session_data_within['delta_giniLocal'] == session_data_within['delta_giniLocal']) & 
    (session_data_within['round'] < 9)
]['session_no']

ols = sm.OLS(y, X)
ols.fit(cov_type='cluster', cov_kwds={'groups': [session_c]}).summary()

### Average inequality

In [ ]:
print(np.mean(data_final_coop.giniGlobal))
print(np.mean(data_final_coop[data_final_coop['giniLocal'] != np.inf].giniLocal))

In [ ]:
data_final_coop.groupby(['round', 'version']).mean()['giniGlobal'].unstack().plot(style = ['-', '--'], 
                                                                                  color = 'black')
plt.xlabel('Round')
plt.ylabel('Gini Index (Global)')
plt.title('Gini Index (Global) by Round')
plt.legend(['1st  PG game', '2nd PG game'])
plt.xticks(np.arange(1, 11, step=1))
plt.show()

In [ ]:
data_final_coop[data_final_coop['giniLocal'] != np.inf].groupby(
    ['round', 'version']).mean()['giniLocal'].unstack().plot(style = ['-', '--'], color = 'black')
plt.xlabel('Round')
plt.ylabel('Gini Index (Local)')
plt.title('Gini Index (Local) by Round')
plt.legend(['1st  PG game', '2nd PG game'])
plt.xticks(np.arange(1, 11, step=1))
plt.show()